In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc

In [ ]:
import celltypist

In [ ]:
import anndata2ri
import logging

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
from rpy2.robjects import pandas2ri
pandas2ri.deactivate()

In [ ]:
%%R
library(Seurat)

In [ ]:
%%R
library(S4Vectors)

In [ ]:
#feel free to change these settings according to your needs
sc.set_figure_params(dpi=80, figsize=(4,4))
sc.settings.verbosity=0
plt.rcParams['figure.dpi'] = 80
plt.rcParams['figure.figsize'] = (4, 4)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
adata=sc.read("adata_version02.h5ad")
adata

In [ ]:
sc.pl.umap(adata, color="annotation_curated")

In [ ]:
#subset endocrine cells
adata_endo=adata[adata.obs["annotation_curated"].isin(["Ngn3 low EP", "Ngn3 high EP", "Fev+", 
                                                       "Beta", "Alpha", "Delta", "Epsilon"])]
adata_endo

In [ ]:
sc.pl.umap(adata_endo, color=["annotation_curated", "day"], wspace=0.5)

In [ ]:
adata_endo.uns["day_colors"]

In [ ]:
adata_endo.obs["day"]

In [ ]:
query=adata_endo[adata_endo.obs["day"].isin(["13.5"])]
query

In [ ]:
reference=adata_endo[~adata_endo.obs["day"].isin(["13.5"])]
reference

In [ ]:
#prepare our training and query dataset so that counts are normalized to 10,000 counts per cell, then log1p-transformed
reference.X=reference.layers["count"].copy()
sc.pp.normalize_per_cell(reference, counts_per_cell_after=10**4)
sc.pp.log1p(reference)

query.X=query.layers["count"].copy()
sc.pp.normalize_per_cell(query, counts_per_cell_after=10**4)
sc.pp.log1p(query)

In [ ]:
custom_model = celltypist.train(reference, labels = 'annotation_curated', n_jobs = 8, feature_selection = True)

In [ ]:
predictions = celltypist.annotate(query, model = custom_model, 
                                  majority_voting = True)

In [ ]:
query_pred = predictions.to_adata()
query_pred

In [ ]:
sc.pl.umap(query_pred, color = ['annotation_curated', 'predicted_labels', 'majority_voting'], wspace=0.4)

In [ ]:
celltypist.dotplot(predictions, use_as_reference = 'annotation_curated', use_as_prediction = 'predicted_labels')

In [ ]:
#make a copy of your anndata object
adata_seurat = adata_endo.copy()

# Convert categorical columns to strings
batch_key="batch"
label_key="annotation_curated"#
adata_seurat.obs[batch_key] = adata_seurat.obs[batch_key].astype(str)
adata_seurat.obs[label_key] = adata_seurat.obs[label_key].astype(str)
# Delete uns as this can contain arbitrary objects which are difficult to convert
del adata_seurat.uns
adata_seurat

In [ ]:
%%R -i adata_seurat
adata_seurat

In [ ]:
%%R -i adata_seurat
seurat <- as.Seurat(adata_seurat, counts = "count", data = "log1p_norm")
seurat

In [ ]:
%%R -i batch_key
batch_list <- SplitObject(seurat, split.by = batch_key)
batch_list

In [ ]:
%%R
anchors <- FindIntegrationAnchors(batch_list, anchor.features = rownames(seurat))
anchors

In [ ]:
%%R
integrated <- IntegrateData(anchors)
integrated

In [ ]:
%%R -o integrated_expr
# Extract the integrated expression matrix
integrated_expr <- GetAssayData(integrated)
# Make sure the rows and columns are in the same order as the original object
integrated_expr <- integrated_expr[rownames(seurat), colnames(seurat)]
# Transpose the matrix to AnnData format
integrated_expr <- t(integrated_expr)
print(integrated_expr[1:10, 1:10])


In [ ]:
adata_seurat.X = integrated_expr
adata_seurat.layers["seurat"] = integrated_expr
print(adata_seurat)
adata.X

In [ ]:
adata_seurat.uns[batch_key + "_colors"] = ['#1f77b4', '#ff7f0e', '#279e68', '#d62728']
sc.tl.pca(adata_seurat)
sc.pp.neighbors(adata_seurat)
sc.tl.umap(adata_seurat)
sc.pl.umap(adata_seurat, color=[label_key, batch_key], wspace=0.5)

In [ ]:
sc.pl.umap(adata_endo, color=["annotation_curated", "day"], wspace=0.5)